In [1]:
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
# from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
import torch
import pickle
torch.cuda.is_available()

True

In [2]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [3]:
from se3_diffuse import rigid_utils as ru
from se3_diffuse import utils as du
from se3_diffuse import se3_diffuser
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import Helix4_Dataset_Score, Make_KNN_MP_Graphs
from gudiff_model.Graph_UNet import GraphUNet
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb

In [4]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

#find better way to incorporate coord_scale

#needed
N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')

In [6]:
def model_score_step(backbone_dict, noised_dict, graph_maker, graph_unet,
                     rot_loss_t_threshold = 0.2, trans_weight=1.0, rot_weight=0.5,
                     device='cuda'):
    
    
    
    batched_t = noised_dict['batched_t'].to(device)
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to(device)
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to(device)
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to(device)
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = noised_dict['CA'].reshape(B, L, 3).to(device)
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to(device)
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to(device)
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)


    pred_trans_score = out['1'][:,0,:]
    pred_rots_score = out['1'][:,1,:]

    true_trans_score = noised_dict['trans_score'].reshape((-1,3)).to(device)
    tss = noisy['trans_score_scaling'][:,None,None].to(device)

    true_rots_score = noised_dict['rot_score'].reshape((-1,3)).to(device)
    rss = noisy['rot_score_scaling'][:,None,None].to(device)


    loss_trans_mse = torch.square(pred_trans_score-true_trans_score)/(tss**2)
    loss_rot_mse = torch.square(pred_rots_score-true_rots_score)/(rss**2)

    rot_loss =   torch.sum(loss_rot_mse,dim=(-1, -2))/ (B*L) #
    trans_loss = torch.sum(loss_trans_mse,dim=(-1, -2))/ (B*L) #
    
    rot_loss *= batched_t > rot_loss_t_threshold

    loss_out = rot_loss*torch.tensor(rot_weight).to(device)+trans_loss*torch.tensor(trans_weight).to(device)
    
    return loss_out

def model_score_step_trans(backbone_dict, noised_dict, graph_maker, graph_unet,
                     rot_loss_t_threshold = 0.2, trans_weight=1.0, rot_weight=0.5,
                     device='cuda'):
    
    
    
    batched_t = noised_dict['batched_t'].to(device)
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to(device)
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to(device)
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to(device)
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = noised_dict['CA'].reshape(B, L, 3).to(device)
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to(device)
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to(device)
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)


    pred_trans_score = out['1'][:,0,:]
    #pred_rots_score = out['1'][:,1,:]

    true_trans_score = noised_dict['trans_score'].reshape((-1,3)).to(device)
    tss = noisy['trans_score_scaling'][:,None,None].to(device)

#     true_rots_score = noised_dict['rot_score'].reshape((-1,3)).to(device)
#     rss = noisy['rot_score_scaling'][:,None,None].to(device)


    loss_trans_mse = torch.square(pred_trans_score-true_trans_score)#/(tss**2)
#     loss_rot_mse = torch.square(pred_rots_score-true_rots_score)/(rss**2)

    #rot_loss =   torch.sum(loss_rot_mse,dim=(-1, -2))/ (B*L) #
    trans_loss = torch.sum(loss_trans_mse,dim=(-1, -2))/ (B*L) #
    trans_loss = torch.clamp(trans_loss, min=None, max=100)
    
    #rot_loss *= batched_t > rot_loss_t_threshold

    loss_out = trans_loss*torch.tensor(trans_weight).to(device)
    
    return loss_out
    

In [7]:
#add threshold and relative weights

#   # Loss weights.
#   trans_loss_weight: 1.0
#   rot_loss_weight: 0.5
#   rot_loss_t_threshold: 0.2
#   separate_rot_loss: True
#   trans_x0_threshold: 1.0
#   coordinate_scaling: ${diffuser.r3.coordinate_scaling}
#   bb_atom_loss_weight: 1.0
#   bb_atom_loss_t_filter: 0.25
#   dist_mat_loss_weight: 1.0
#   dist_mat_loss_t_filter: 0.25
#   aux_loss_weight: 0.25

In [8]:
# rot_mse = (gt_rot_score - pred_rot_score)**2 * loss_mask[..., None]
# rot_loss = torch.sum(
#     rot_mse / rot_score_scaling[:, None, None]**2,
#     dim=(-1, -2)
# ) / (loss_mask.sum(dim=-1) + 1e-10)
# rot_loss *= self._exp_conf.rot_loss_weight
# rot_loss *= batch['t'] > self._exp_conf.rot_loss_t_threshold

In [9]:
class EMA(nn.Module):
    def __init__(self, mu):
        super(EMA, self).__init__()
        self.mu = mu
        self.shadow = {}

    def register(self, name, val):
        self.shadow[name] = val.clone()

    def forward(self, name, x):
        assert name in self.shadow
        new_average = (1.0 - self.mu) * x + self.mu * self.shadow[name]
        self.shadow[name] = new_average.clone()
        return new_average



In [10]:

# in batch training loop
# for batch in batches:

# optimizer.step()
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         param.data = ema(name, param.data)

In [11]:
def make_save_folder(name=''):
    base_folder = time.strftime(f'log/%y%b%d_%I%M%p_{name}/', time.localtime())
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    subfolders = ['models']
    for subfolder in subfolders:
        if not os.path.exists(base_folder + subfolder):
            os.makedirs(base_folder + subfolder)
            
    return base_folder
        
def save_chkpt(model_path, model, optimizer, epoch, batch, val_losses, train_losses):
    """Save a training checkpoint
    Args:
        model_path (str): the path to save the model to
        model (nn.Module): the model to save
        optimizer (torch.optim.Optimizer): the optimizer to save
        epoch (int): the current epoch
        batch (int): the current batch in the epoch
        loss_domain (list of int): a list of the shared domain for val and training 
            losses
        val_losses (list of float): a list containing the validation losses
        train_losses (list of float): a list containing the training losses
    """
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    state_dict = dict()
    state_dict.update({'model':model.state_dict(),
                       'optimizer':optimizer.state_dict(),
                       'epoch':epoch,
                       'batch':batch,
                       'train_losses':train_losses,
                       'val_losses':val_losses
                       })
    torch.save(state_dict, f'{model_path}model_e{epoch}')

    
def save_chkpt_ema(model_path, model, optimizer, epoch, batch, ema, train_losses):
    """Save a training checkpoint
    Args:
        model_path (str): the path to save the model to
        model (nn.Module): the model to save
        optimizer (torch.optim.Optimizer): the optimizer to save
        epoch (int): the current epoch
        batch (int): the current batch in the epoch
        loss_domain (list of int): a list of the shared domain for val and training 
            losses
        val_losses (list of float): a list containing the validation losses
        train_losses (list of float): a list containing the training losses
    """
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    state_dict = dict()
    state_dict.update({'model':model.state_dict(),
                       'optimizer':optimizer.state_dict(),
                       'epoch':epoch,
                       'batch':batch,
                       'train_losses':train_losses
                       })
    torch.save(state_dict, f'{model_path}model_e{epoch}')

    # Open a file and use dump()
    with open(f'{model_path}/ema_save_{epoch}.pkl', 'wb') as file:
        # A new file will be created
        pickle.dump(ema, file)
    
def load_model(model_path, model_class):
    """Load a saved model"""
    
    device = 'cuda:0'
    model = model_class()
    model.load_state_dict(torch.load(model_path)['model'])
    model.to(device)
    
    return model 
    
def load_model_ema(model_path, model_class, e):
    """Load a saved model"""
    
    model_path_m = f'{model_path}/model_e{e}'
    
    device = 'cuda:0'
    model = model_class(batch_size = B, num_layers_ca = 2, zero_lin=False)
    model.load_state_dict(torch.load(model_path_m)['model'])
    model.to(device)
    opti = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=5e-5)
    opti.load_state_dict(torch.load(f'{model_path_m}')['optimizer'])
    
    with open(f'{model_path}/ema_save_{e}.pkl', 'rb') as file:
        # Call load method to deserialze
        ema_out = pickle.load(file)

    return model, ema_out, opti

In [12]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for val/train split
#tog = together helices for val/train split
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 5048
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [18]:
B = 32
L=65
limit = 2028
h4_trainData = Helix4_Dataset_Score(coords_tog[:limit])
train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
test_iter = iter(train_dL)
test_batch = next(test_iter)

In [19]:
se3d = se3_diffuser.SE3Diffuser()

In [20]:
gu = GraphUNet(batch_size = B, num_layers_ca = 2, zero_lin=False).to('cuda')
opti = torch.optim.Adam(gu.parameters(), lr=0.0005, weight_decay=5e-5)
ema = EMA(0.980)
for name, param in gu.named_parameters():
    if param.requires_grad:
        ema.register(name, param.data)
gm = Make_KNN_MP_Graphs() #consider precalculating graphs for training

In [16]:
# model_path = 'log/23Sep11_0841AM_full_diff_score_trans_only'
# e_start=190
# gu, ema, opti = load_model_ema(model_path, GraphUNet, 189)
#ema.mu = 0.95
# num_epochs = 200
# e_start= 190
# #model_path = make_save_folder(name=f'full_diff_score_trans_only')
# save_per=10
# avg_vloss=0
# avg_tloss=0

# for e in range(e_start, e_start+num_epochs):
    
#     running_tloss = 0 
#     start = time.time()
#     for i, bb_dict in enumerate(train_dL):
#         noisy = se3d.forward_marginal_trans(bb_dict,t_vec=None)
#         train_loss = torch.sum(model_score_step_trans(bb_dict, noisy, gm, gu))

#         opti.zero_grad()
#         train_loss.backward()
#         opti.step()
        
#         for name, param in gu.named_parameters():
#             if param.requires_grad:
#                 param.data = ema(name, param.data)

#         running_tloss += train_loss.detach().cpu()
    
#     end = time.time()
#     avg_tloss = running_tloss/(i+1)
#     print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')
    
    
#     if e %save_per==save_per-1:
#         save_chkpt_ema(f'{model_path}', gu, opti, e, B, ema, avg_tloss)

In [17]:
# se3d._diffuse_rot = False
# test_iter = iter(train_dL)
# test_batch = next(test_iter)
# dt = 0.01
# t_start = 0.01
# t_end = 0
# tr = np.arange(t_start,0,-dt)
# tr
# t_vec = np.ones((B,))*t_start
# noisy = se3d.forward_marginal_trans(test_batch,t_vec=t_vec)
# batched_t = noisy['batched_t'].to('cuda')
# x = gm.prep_for_network(noisy)
# out = gu(x, batched_t)
# ts = out['1'][:,0,:].reshape((32,65,3))
# trans_t_1 = se3d._r3_diffuser.reverse(
#                 x_t=noisy['CA'].numpy(),
#                 score_t=ts.detach().cpu().numpy(),
#                 t=t_start,
#                 dt=t_start,
#                 center="True",
#                 noise_scale=0
#                 )
# device='cpu'
# CA_n  = trans_t_1.reshape(B, L, 3)
# NC_n = CA_n + noisy['N_CA'].reshape(B, L, 3).numpy()
# CC_n = CA_n + noisy['C_CA'].reshape(B, L, 3).numpy()
# noise_xyz =  np.concatenate((NC_n,CA_n,CC_n),axis=2).reshape(B,L,3,3)
# dump_coord_pdb(noise_xyz[0]*10,fileOut='output/test.pdb')

In [ ]:
#fresh run
num_epochs = 200
e_start= 0
model_path = make_save_folder(name=f'full_diff_score_trans_only_notss')
save_per=10
avg_vloss=0
avg_tloss=0

for e in range(e_start, e_start+num_epochs):
    
    running_tloss = 0 
    start = time.time()
    for i, bb_dict in enumerate(train_dL):
        noisy = se3d.forward_marginal_trans(bb_dict,t_vec=None)
        train_loss = torch.sum(model_score_step_trans(bb_dict, noisy, gm, gu))

        opti.zero_grad()
        train_loss.backward()
        opti.step()
        
        for name, param in gu.named_parameters():
            if param.requires_grad:
                param.data = ema(name, param.data)

        running_tloss += train_loss.detach().cpu()
    
    end = time.time()
    avg_tloss = running_tloss/(i+1)
    print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')
    
    
    if e %save_per==save_per-1:
        save_chkpt_ema(f'{model_path}', gu, opti, e, B, ema, avg_tloss)
    

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [16]:
num_epochs = 200
e_start= 0
model_path = make_save_folder(name=f'full_diff_score_t_only')
save_per=10
avg_vloss=0
avg_tloss=0

for e in range(e_start, e_start+num_epochs):
    
    running_tloss = 0 
    start = time.time()
    for i, bb_dict in enumerate(train_dL):
        noisy = se3d.forward_marginal(bb_dict,t_vec=None)
        train_loss = torch.sum(model_score_step(bb_dict, noisy, gm, gu))

        opti.zero_grad()
        train_loss.backward()
        opti.step()

        running_tloss += train_loss.detach().cpu()
    
    end = time.time()
    avg_tloss = running_tloss/(i+1)
    print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')
    
    
    if e %save_per==save_per-1:
        save_chkpt(f'log/{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)
    
    

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Average Train Loss Epoch 0: 3084.808837890625;   Epoch time: 245
Average Train Loss Epoch 1: 3649.027587890625;   Epoch time: 249
Average Train Loss Epoch 2: 2278.03173828125;   Epoch time: 245
Average Train Loss Epoch 3: 4651.07763671875;   Epoch time: 244
Average Train Loss Epoch 4: 3042.232666015625;   Epoch time: 242
Average Train Loss Epoch 5: 3258.308349609375;   Epoch time: 242
Average Train Loss Epoch 6: 10177.689453125;   Epoch time: 241
Average Train Loss Epoch 7: 2328.85546875;   Epoch time: 241
Average Train Loss Epoch 8: 4874.81982421875;   Epoch time: 241
Average Train Loss Epoch 9: 2034.1339111328125;   Epoch time: 241
Average Train Loss Epoch 10: 5845.41015625;   Epoch time: 241
Average Train Loss Epoch 11: 4438.75927734375;   Epoch time: 241
Average Train Loss Epoch 12: 4779.22314453125;   Epoch time: 242
Average Train Loss Epoch 13: 2697.35595703125;   Epoch time: 242
Average Train Loss Epoch 14: 3766.337646484375;   Epoch time: 242
Average Train Loss Epoch 15: 6226.7

Average Train Loss Epoch 127: 2368.1220703125;   Epoch time: 240


KeyboardInterrupt: 